## Langfuse LLM Observability and Prompt Management Tutorial

### What is Langfuse?
Langfuse is a powerful platform for:

* LLM Observability: Track and monitor LLM interactions in production
* Tracing: Detailed execution traces for debugging and optimization
* Analytics: Performance metrics and cost tracking
* Evaluation: Automated quality assessment of LLM outputs

In [ ]:
%pip install langfuse langchain langchain-google-genai --upgrade

### Environment Configuration

In [ ]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "YOUR_API_KEY"
os.environ["LANGFUSE_SECRET_KEY"] = "YOUR_API_KEY"
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Your api_key
os.environ["GOOGLE_API_KEY "] = "YOUR_API_KEY"

Purpose: Set up authentication for both Langfuse and ChatGoogleGenerativeAI services.
⚠️ Security Note: In production, use environment variables or secure credential management instead of hardcoding API keys

### Initialize Langfuse Client

In [ ]:
from langfuse import get_client
from langfuse.langchain import CallbackHandler

# Initialize Langfuse client (prompt management)
langfuse = get_client()

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_callback_handler = CallbackHandler()

### Prompt Management - Create Prompt Template

In [ ]:
langfuse.create_prompt(
    name="event-planner",
    prompt=
    "Plan an event titled {{Event Name}}. The event will be about: {{Event Description}}. "
    "The event will be held in {{Location}} on {{Date}}. "
    "Consider the following factors: audience, budget, venue, catering options, and entertainment. "
    "Provide a detailed plan including potential vendors and logistics.",
    config={
        "model":"gpt-4o",
        "temperature": 0,
    },
    labels=["production"]
);

### Retrieve and Use Prompt

In [ ]:
# Get current production version of prompt
langfuse_prompt = langfuse.get_prompt("event-planner")

In [ ]:
print(langfuse_prompt.prompt)

Plan an event titled {{Event Name}}. The event will be about: {{Event Description}}. The event will be held in {{Location}} on {{Date}}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

langchain_prompt = ChatPromptTemplate.from_template(
        langfuse_prompt.get_langchain_prompt(),
        metadata={"langfuse_prompt": langfuse_prompt},
    )

### Model Configuration from Prompt

In [ ]:
model = langfuse_prompt.config["model"]
temperature = str(langfuse_prompt.config["temperature"])
print(f"Prompt model configurations\nModel: {model}\nTemperature: {temperature}")

Prompt model configurations
Model: gpt-4o
Temperature: 0


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)

chain = langchain_prompt | model

### Execute with Tracing

In [ ]:
example_input = {
    "Event Name": "Wedding",
    "Event Description": "The wedding of Julia and Alex, a charming couple who share a love for art and nature. This special day will celebrate their journey together with a blend of traditional and contemporary elements, reflecting their unique personalities.",
    "Location": "Central Park, New York City",
    "Date": "June 5, 2024"
}

In [ ]:
# we pass the callback handler to the chain to trace the run in Langfuse
response = chain.invoke(input=example_input,config={"callbacks":[langfuse_callback_handler]})

print(response.content)

To plan an event titled "{Event Name}" in Central Park, New York City on June 5, 2024, we need to consider several key factors: audience, budget, venue, catering options, and entertainment. Below is a detailed plan:

### Event Overview
- **Event Name**: {Event Name}
- **Date**: June 5, 2024
- **Location**: Central Park, New York City
- **Description**: {Event Description}

### Audience
- **Target Audience**: Define the demographic (e.g., families, young professionals, industry experts, etc.)
- **Expected Attendance**: Estimate the number of attendees (e.g., 200-500 people)

### Budget
- **Total Budget**: Determine the overall budget (e.g., $50,000)
- **Allocation**:
  - Venue: 20%
  - Catering: 25%
  - Entertainment: 20%
  - Marketing: 15%
  - Logistics and Rentals: 10%
  - Miscellaneous: 10%

### Venue
- **Location**: Central Park, NYC
- **Permits**: Obtain necessary permits from NYC Parks Department
- **Layout**: Designate areas for stage, seating, food stalls, and restrooms
- **Logi